<a href="https://colab.research.google.com/github/Tiabet/Project_Market/blob/master/BertForSequence%2BAutoTokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00


In [26]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

inputs = tokenizer("끔찍해요", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits
    #probabilities = torch.softmax(logits, dim=1) #가능성까지 확인하고 싶으면

predicted_class_id = logits.argmax().item()
print(predicted_class_id) #0이면 긍정, 1이면 부정
#print(probabilities)
#print(model.config.id2label[predicted_class_id])


1


In [12]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")


('ratings_total.txt', <http.client.HTTPMessage at 0x7ec2a73ea500>)

In [14]:
import pandas as pd

total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력


전체 리뷰 개수 : 200000


In [19]:
total_data.drop(columns = ['ratings'])
total_data = total_data.sample(n=1000, random_state =42)

In [18]:
total_data

,ratings,reviews
142061,4,효과는 좋은데 .. 배송이은 별로! 배송조회가 안된다는 점이 너무 불편해요 . 언제...
41119,1,횟감용 선도가 아닙니다. 냄새 납니다. 구워먹자니 너무 많아서 개나 구워줘야 겟어요...
55087,4,콧물같이 너무 미끄덩 거리지만 색소 안들어있어 좋습니다
98846,1,끓는 시간 엄청 느려요;
71330,5,정말 맛나요 ㅓ저대니ㅣ니니느우ㅜㅇ퉈차라아ㅏ내재자누ㅜ너너나ㅜ어야애ㅐ대ㅏ저너ㅜ우나니나다...
...,...,...
67737,5,너무나도 시원한 티셔츠!!!
142669,5,이틀째 시용했는데 10분사용하고나면 광대쪽이 올라가는듯 좀 탱탱한 느낌이 있네요ㅋㅋ...
64177,5,재구매 늘 오던 그제품 그대로입니다 냉동팩 및 포장상태도 좋고 다만.. 지난번보다 ...
147432,5,아주좋아요 아주좋네요 후기대로 고급자 그레이 빛이 납니다 깔끔하고 길이도 잘맞네요 ...


In [28]:
reviews = total_data['reviews']

In [32]:
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

pred_result = []
for review in reviews :
  inputs = tokenizer(review, return_tensors="pt")

  with torch.no_grad():
      logits = model(**inputs).logits
      #probabilities = torch.softmax(logits, dim=1) #가능성까지 확인하고 싶으면

  predicted_class_id = logits.argmax().item()
  pred_result.append(predicted_class_id)
  #print(predicted_class_id) #0이면 긍정, 1이면 부정
  #print(probabilities)
  #print(model.config.id2label[predicted_class_id])


In [34]:
total_data['Pos/Neg'] = pred_result
total_data

,ratings,reviews,Pos/Neg
68666,2,넘 이름만 보고 샀나봐요.ㅜㅜ 펄이 파티라도 가야할판이네요,0
34694,2,아기넣기너무힘들어요,1
82223,1,제철이 아닌가요 알이 너무 작네요,0
12144,5,"강아지가 잘먹어요, , ,",0
26821,5,운동할때 잘 쓰고 있습니다^^,1
...,...,...,...
42600,1,배송은 빠르나 제품이 누락 및 오배송됨 환불기간 오래걸림,0
101726,1,다 좋았는데 부품이 부서져서왔는데 공급사에서 전체 반품 또는 교환을 해준다니 거기에...,0
143938,2,너무 짜서.. 못먹겟어요,1
145220,1,그냥 그래여 잔향이 별로이여,1


In [38]:
total_data = total_data.drop(columns = ['ratings'])

In [39]:
total_data.to_csv("reviews_predicted.csv",index = False)